In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import glob
import os
import warnings
import numpy as np
from bokeh.io import export_svgs, output_notebook
from bokeh.models import BoxAnnotation, ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from utils.measuring_performance import (
    get_prediction,
    plot_confusion_matrix,
    plot_histogram_by_class,
    plot_loss_per_epoch,
    plot_pr_curve,
    plot_roc_curve,
)
from utils.misc import build_files_list, dump_pickle, load_pickle
from utils.sound_utils import extract_signal_features, generate_dataset, load_sound_file

output_notebook()
warnings.filterwarnings("ignore")
np.random.seed(42)

Loading BokehJS ...

In [18]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib

tf.set_random_seed(42)

In [19]:
def get_available_gpus():
    local_devices = device_lib.list_local_devices()
    return [x.name for x in local_devices if x.device_type == "GPU"]

The boolean variable `COMBINE_MACHINE_ID` allows you to decide whether to perform modeling separately for each machine ID, or combine all data from the same machine type to perform modeling.

In [20]:
DATA_PATH = "../mimii-anomaly-detection"
IMAGE_PATH = "./img"
MODEL_PATH = "./models"
MERGE_MACHINE_ID = True

os.makedirs(os.path.join(DATA_PATH, "dataset"), exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

file_paths = sorted(
    glob.glob(DATA_PATH + "/*/*" if MERGE_MACHINE_ID else DATA_PATH + "/*/*/*")
)

# Data Splitting and Preprocessing

In [21]:
file_path = file_paths[0]
file_path = os.path.normpath(file_path)
file_path_split = file_path.split(os.path.sep)
SUFFIX = "_" + file_path_split[1]

if MERGE_MACHINE_ID:
    print(f"db: {file_path_split[-2]}, machine type: {file_path_split[-1]}")

else:
    print(
        f"db: {file_path_split[-3]}, machine type: {file_path_split[-2]}, machine id: {file_path_split[-1]}"
    )
    SUFFIX = "_".join([SUFFIX, file_path_split[-3]])

db: dataset2, machine type: test_files__mimii-anomaly-detection.txt


Because unsupervised learning is used, all anomalous labels are assigned to the test set.

In [22]:
train_files, test_files, train_labels, test_labels = [], [], [], []

for path in file_paths:
    normal_files, abnormal_files = build_files_list(path)
    normal_labels = np.zeros(len(normal_files))
    abnormal_labels = np.ones(len(abnormal_files))

    # Split normal files for training and testing
    train_files_split, test_files_split, train_labels_split, test_labels_split = train_test_split(
        normal_files, normal_labels, train_size=0.7, random_state=42, shuffle=True
    )

    # Extend training set
    train_files.extend(train_files_split)
    train_labels.extend(train_labels_split)

    # Extend testing set with remaining normal files and all abnormal files
    test_files.extend(test_files_split)
    test_labels.extend(test_labels_split)
    test_files.extend(abnormal_files)
    test_labels.extend(abnormal_labels)

# Shuffle test set indices
test_indices = np.arange(len(test_files))
np.random.shuffle(test_indices)

# Reorder test files and labels
test_files = np.array(test_files)[test_indices]
test_labels = np.array(test_labels)[test_indices]

print(
    f"Train set has {len(train_files)} signals including {int(sum(train_labels))} abnormal signals, \
    while test set has {len(test_files)} signals including {int(sum(test_labels))} abnormal signals."
)

Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset2\test_files__mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset2\test_labels__mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset2\train_data__mimii-anomaly-detection.pkl
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset2\train_files__mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset2\train_labels__mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset\test_files_mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset\test_labels_mimii-anomaly-detection.txt
Found 0 normal files and 0 abnormal files in ../mimii-anomaly-detection\dataset\train_data_mimii-anomaly-detection.pkl
Found 0 normal files and 0 abnorm

In [23]:
dataset = {
    "train_files": train_files,
    "test_files": test_files,
    "train_labels": train_labels,
    "test_labels": test_labels,
}

for key, values in dataset.items():
    file_name = os.path.join(DATA_PATH, "dataset", key + SUFFIX + ".txt")
    with open(file_name, "w") as f:
        for item in values:
            f.write(str(item) + "\n")

In [24]:
n_fft = 2048
hop_length = 512
n_mels = 64
frames = 5

train_data_path = os.path.join(DATA_PATH, "dataset", "train_data" + SUFFIX + ".pkl")

if os.path.exists(train_data_path):
    print("Train data already exists, loading from file...")
    train_data = load_pickle(train_data_path)

else:
    train_data = generate_dataset(
        train_files, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )
    print("Saving train data to disk...")
    dump_pickle(train_data_path, train_data)
    print("Done.")

print(f"Train data has a {train_data.shape} shape.")

Train data already exists, loading from file...
Train data has a (810507, 320) shape.


# Model Training and Prediction with AutoEncoder

In [25]:
def autoencoder(input_dims, model_name=None):
    input_layer = Input(shape=(input_dims,))
    output = Dense(64, activation="relu")(input_layer)
    output = Dense(64, activation="relu")(output)
    output = Dense(8, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(input_dims, activation=None)(output)

    return Model(inputs=input_layer, outputs=output, name=model_name)

In [26]:
MODEL_NAME = "AutoEncoder"
model = autoencoder(n_mels * frames, model_name=MODEL_NAME)
print(model.summary())

gpu_count = len(get_available_gpus())
if gpu_count > 1:
    model = multi_gpu_model(model, gpus=gpu_count)

Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                20544     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_9 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 320)               

In [27]:
%%time
batch_size = 1024
epochs = 400

model.compile(
    optimizer=Adam(learning_rate=1e-03),
    loss="mean_squared_error"
)

history = model.fit(
    train_data,
    train_data,
    batch_size=batch_size,
    epochs=epochs,
    verbose=False,
    callbacks=[EarlyStopping(monitor="val_loss", patience=10)],
    validation_split=0.1,
    shuffle=True
)

model.save(os.path.join(MODEL_PATH, MODEL_NAME + SUFFIX + ".h5"))

Wall time: 10min 28s


In [28]:
plot_loss_per_epoch(
    history, model_name=MODEL_NAME, file_name=os.path.join(IMAGE_PATH, "model_loss.svg")
)

In [29]:
recon_errors = []

for index in tqdm(range(len(test_files))):
    signal, sr = load_sound_file(test_files[index])

    features = extract_signal_features(
        signal, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )

    predictions = model.predict(features)
    mse = np.mean(np.mean(np.square(features - predictions), axis=1))
    recon_errors.append(mse)

  0%|          | 0/1582 [00:00<?, ?it/s]

# Model Evaluation

In [30]:
stack = np.column_stack((range(len(recon_errors)), recon_errors))
score_false = stack[test_labels == 0][:, 1]
score_true = stack[test_labels == 1][:, 1]

plot_histogram_by_class(
    score_false,
    score_true,
    bins=[20, 30],
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "recon_error_dist.svg"),
)

In [34]:
THRESHOLD_MIN = 2.0
THRESHOLD_MAX = 7.0

p = figure(
    plot_width=600,
    plot_height=400,
    title=f"{MODEL_NAME}: Threshold Range Exploration",
    x_axis_label="Samples",
    y_axis_label="Reconstruction Error",
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 0][:, 0], error=stack[test_labels == 0][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="crimson",
    line_color=None,
    legend_label="Normal Signals",
    source=source,
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 1][:, 0], error=stack[test_labels == 1][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="indigo",
    line_color=None,
    legend_label="Abnormal Signals",
    source=source,
)

source = ColumnDataSource(
    data=dict(
        index=stack[:, 0],
        threshold_min=np.repeat(THRESHOLD_MIN, stack.shape[0]),
        threshold_max=np.repeat(THRESHOLD_MAX, stack.shape[0]),
    )
)

box = BoxAnnotation(
    bottom=THRESHOLD_MIN,
    top=THRESHOLD_MAX,
    fill_alpha=0.1,
    fill_color="magenta",
    line_color="darkmagenta",
    line_width=1.0,
)
p.add_layout(box)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_right"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(HoverTool(tooltips=[("index", "@index"), ("error", "@error")]))
show(p)

p.output_backend = "svg"
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, "thr_range_exp.svg"))

In [35]:
THRESHOLD_STEP = 0.2
thresholds = np.arange(THRESHOLD_MIN, THRESHOLD_MAX + THRESHOLD_STEP, THRESHOLD_STEP)
errors = []

for threshold in thresholds:
    predictions = get_prediction(stack[:, 1], threshold=threshold)
    conf_mat = confusion_matrix(test_labels, predictions)
    errors.append([threshold, conf_mat[1, 0], conf_mat[0, 1]])

errors = np.array(errors)

In [36]:
p = figure(
    plot_width=600,
    plot_height=400,
    title=f"{MODEL_NAME}: Best Threshold Exploration",
    x_axis_label="Reconstruction Error Threshold (%)",
    y_axis_label="# Samples",
)

source = ColumnDataSource(
    data=dict(
        threshold=errors[:, 0], false_negative=errors[:, 1], false_positive=errors[:, 2]
    )
)

p.line(
    x="threshold",
    y="false_negative",
    color="crimson",
    legend_label="False Negative",
    source=source,
)
p.circle(
    x="threshold", y="false_negative", color="crimson", fill_color="white", source=source
)
p.line(
    x="threshold",
    y="false_positive",
    color="indigo",
    legend_label="False Positive",
    source=source,
)
p.circle(
    x="threshold", y="false_positive", color="indigo", fill_color="white", source=source
)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(
    HoverTool(
        tooltips=[
            ("threshold", "@threshold"),
            ("false_negative", "@false_negative"),
            ("false_positive", "@false_positive"),
        ]
    )
)
show(p)

p.output_backend = "svg"
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, "best_thr_exp.svg"))

In [37]:
THRESHOLD = 3.6
predictions = get_prediction(stack[:, 1], threshold=THRESHOLD)

plot_confusion_matrix(
    confusion_matrix(test_labels, predictions),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "conf_mat.svg"),
)

print(
    f"Accuracy: {accuracy_score(test_labels, predictions):.2%}, \
Precision: {precision_score(test_labels, predictions):.2%}, \
Recall: {recall_score(test_labels, predictions):.2%}, \
F1: {f1_score(test_labels, predictions):.2%}"
)

Accuracy: 42.41%, Precision: 31.70%, Recall: 86.40%, F1: 46.38%


In [38]:
plot_roc_curve(
    roc_curve(test_labels, recon_errors),
    roc_auc_score(test_labels, recon_errors),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "roc_curve.svg"),
)

In [39]:
plot_pr_curve(
    precision_recall_curve(test_labels, recon_errors),
    average_precision_score(test_labels, recon_errors),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "pr_curve.svg"),
)